# AppBuilder-Knowledge：生产环境的知识库/文档/切片管理教学

[知识库组件](https://github.com/baidubce/app-builder/blob/master/docs/basic_module/knowledgebase.md)

知识库组件（KnowledgeBase）是对线上知识库操作的组件，可以通过SDK实现创建知识库、添加知识文档、查询知识库文档、删除知识文档等操作，可在平台console中查看结果。对console端知识库进行操作，可以通过SDK实现创建知识库、添加知识文档、查询知识库文档、删除知识文档等操作，可在平台console中查看结果

应用场景：通过SDK代码实现console端知识库操作。

#### AppBuilder-Knowledge代码态使用流程

##### 新建一个知识库

- 首先我们需要通过SDK新建一个知识库，新建知识库的代码示例如下：

In [52]:
import os
import appbuilder

os.environ["APPBUILDER_TOKEN"] = "bce-v3/ALTAK-n5AYUIUJMarF7F7iFXVeK/1bf65eed7c8c7efef9b11388524fa1087f90ea58"

knowledge = appbuilder.KnowledgeBase()
resp = knowledge.create_knowledge_base(
        name="my_knowledge",
        description="my_knowledge",
        type="public",
    )
print("新建的知识库ID: ", resp.id)
print("新建的知识库名称: ", resp.name)

新建的知识库ID:  3cc48493-f9ff-48c8-a3ea-fcc17fdddded
新建的知识库名称:  my_knowledge


##### 实例化已创建的知识库 
- 接下来如果我们期望对知识库进行进一步操作,就需要实例化已创建的知识库 

In [53]:
my_knowledge_base_id = resp.id # 传入知识库ID
my_knowledge = appbuilder.KnowledgeBase(my_knowledge_base_id)
print("知识库ID: ", my_knowledge.knowledge_id)

知识库ID:  3cc48493-f9ff-48c8-a3ea-fcc17fdddded


- 我们就成功在Appbuilder平台创建了一个名为my_knowledge的知识库

![创建知识库](https://bj.bcebos.com/v1/appbuilder-sdk-components/%E5%88%9B%E5%BB%BA%E7%9F%A5%E8%AF%86%E5%BA%93.png?authorization=bce-auth-v1%2FALTAKGa8m4qCUasgoljdEDAzLm%2F2024-08-22T02%3A02%3A57Z%2F-1%2Fhost%2F0134e32c4f9221cfd3cb8ff74737ea000079639342b05fe4649a07fd9b18c11c)

##### 获取知识库详情
- 如果我们想要获取知识库详情，可以调用 `get_knowledge_base_detail` 方法,返回的response将包含知识库的详情信息

In [54]:
knowledge = appbuilder.KnowledgeBase()
resp = knowledge.get_knowledge_base_detail(my_knowledge.knowledge_id)
print("查询的知识库ID: ", resp.id)
print("查询的知识库名称: ", resp.name)

查询的知识库ID:  3cc48493-f9ff-48c8-a3ea-fcc17fdddded
查询的知识库名称:  my_knowledge


##### 获取知识库列表
- 当你想查看你的账号的知识库，SDK同样支持调用知识库列表接口，获取知识库列表。需要提供知识库id作为起始搜索位置，不提供则会默认从第一个知识库开始搜索

In [55]:
knowledge = appbuilder.KnowledgeBase()
# 从my_knowledge知识库开始检索，检索10条
resp = knowledge.get_knowledge_base_list(my_knowledge.knowledge_id,10)
print("获取到的知识库列表: ", resp)

获取到的知识库列表:  requestId='8e6d9bd4-73c2-41b9-9813-de65a5ca01ec' data=[KnowledgeBaseDetailResponse(id='271bba8b-2c77-4b52-aaa2-2d95dde9184a', name='千帆大模型平台文档', description=None, config=KnowledgeBaseConfig(index=KnowledgeBaseConfigIndex(type='public', esUrl='', username=None, password=None)))] marker='3cc48493-f9ff-48c8-a3ea-fcc17fdddded' nextMarker='271bba8b-2c77-4b52-aaa2-2d95dde9184a' maxKeys=1 isTruncated=False


##### 修改知识库
- 如果我们对我们创建的知识库不满意，想要对其修改，SDK提供`modify_knowledge_base`方法，可以修改知识库,在下面的示例中我们将展示如何修改知识库的名称，接着我们按照知识库的id，查询知识库，查询知识库的名称是否改变,当然我们也可以去appbuilder平台个人空间去查看知识库名称是否更新


In [56]:
knowledge = appbuilder.KnowledgeBase()
knowledge.modify_knowledge_base(my_knowledge.knowledge_id, name="new_test_knowledge", description="测试")

# 接着我们按照知识库的id，查询知识库，查询知识库的名称是否改变
resp = knowledge.get_knowledge_base_detail(my_knowledge.knowledge_id)
print("查询的知识库ID: ", resp.id)
print("查询的知识库名称: ", resp.name)

查询的知识库ID:  3cc48493-f9ff-48c8-a3ea-fcc17fdddded
查询的知识库名称:  new_test_knowledge


##### 上传文档到知识库
上传文档到知识库共有以下三种方式：
- 上传文档到知识库
- 上传通用文档
- 向知识库添加文档

获取目标文档的file_url

In [57]:
file_url = "https://bj.bcebos.com/v1/appbuilder-sdk-components/%E3%80%8A%E9%BB%91%E7%A5%9E%E8%AF%9D%EF%BC%9A%E6%82%9F%E7%A9%BA%E3%80%8BIP%E4%BB%8B%E7%BB%8D%E6%89%8B%E5%86%8C2024.pdf?authorization=bce-auth-v1%2FALTAKGa8m4qCUasgoljdEDAzLm%2F2024-08-21T11%3A40%3A45Z%2F-1%2Fhost%2Ff387e2cd2db95c9da26247c5d6c45f6dc9a817fa2721f0a780dc1eba433d02b8"

###### 上传文档到知识库
- 主要提供自定义文档处理策略，向知识库添加文档
  - 文档格式：rawText (允许配置后续分割策略), qa(不支持配置后续分割策略)
  - 文档处理策略

In [58]:
knowledge_base_id = my_knowledge.knowledge_id
knowledge.create_documents(
	id=knowledge_base_id,
	contentFormat="rawText",
	source=appbuilder.DocumentSource(
		type="web",
		urls=["https://baike.baidu.com/item/%E9%BB%91%E7%A5%9E%E8%AF%9D%EF%BC%9A%E6%82%9F%E7%A9%BA/53303078?fr=ge_ala"],
		urlDepth=1,
	),
	processOption=appbuilder.DocumentProcessOption(
		template="custom",
		parser=appbuilder.DocumentChoices(
			choices=["layoutAnalysis", "ocr"]
		),
		chunker=appbuilder.DocumentChunker(
			choices=["separator"],
			separator=appbuilder.DocumentSeparator(
				separators=["。"],
				targetLength=300,
				overlapRate=0.25,
			),
			prependInfo=["title", "filename"],
		),
		knowledgeAugmentation=appbuilder.DocumentChoices(choices=["faq"]),
	),
)

{'requestId': 'f698099c-b533-4500-a1f8-6b9f2087d1ba'}

###### 上传通用文档
- SDK支持基于代码态的文档上传方法`upload_file`,以及包含文档的自定义切分逻辑的向知识库添加文档的方法``add_document`
- 需要先将文档下载至本地

In [59]:
import requests

def download_file(url, file_path):
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()
        with open(file_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    file.write(chunk)
        print(f"文件已成功保存到 {file_path}")
    except requests.RequestException as e:
        print(f"下载失败: {e}")

file_url = "https://bj.bcebos.com/v1/appbuilder-sdk-components/%E3%80%8A%E9%BB%91%E7%A5%9E%E8%AF%9D%EF%BC%9A%E6%82%9F%E7%A9%BA%E3%80%8BIP%E4%BB%8B%E7%BB%8D%E6%89%8B%E5%86%8C2024.pdf?authorization=bce-auth-v1%2FALTAKGa8m4qCUasgoljdEDAzLm%2F2024-08-21T11%3A40%3A45Z%2F-1%2Fhost%2Ff387e2cd2db95c9da26247c5d6c45f6dc9a817fa2721f0a780dc1eba433d02b8"
file_path = "黑神话(悟空).pdf"  # 替换为你想要的文件名

# 调用函数下载文件
download_file(file_url, file_path)

文件已成功保存到 黑神话(悟空).pdf


In [60]:
upload_res = knowledge.upload_file(file_path)
add_res = knowledge.add_document(
    content_type="raw_text",
    file_ids=[upload_res.id],
    custom_process_rule=appbuilder.CustomProcessRule(
        separators=["?"], target_length=400, overlap_rate=0.2
    ),
    knowledge_base_id=my_knowledge.knowledge_id
)

##### 获取知识库的文档列表
在完成添加文档之后，如果我们想获取知识库中的文档列表，可以使用下面的方法：
- 获取知识库的文档列表:`get_documents_list` 单次请求列表获得的文档数量,最大100
- 获取知识库全部文档:`get_all_documents`

In [61]:
my_knowledge = appbuilder.KnowledgeBase(my_knowledge_base_id)
print("知识库ID: ", my_knowledge.knowledge_id)

# 获取知识库的文档列表:get_documents_list
list_res = my_knowledge.get_documents_list()
print("get_documents_list")
print("文档列表: ", list_res)

# 获取知识库全部文档:get_all_documents
doc_list = knowledge.get_all_documents(my_knowledge.knowledge_id)
print("get_all_documents")
for message in doc_list:
    print(message)

知识库ID:  3cc48493-f9ff-48c8-a3ea-fcc17fdddded
get_documents_list
文档列表:  request_id='da7bc5a5-c42d-47cd-82e7-674ff3d65502' data=[Document(id='139be427-5024-4af7-b5ca-1f15efdfea86', name='黑神话(悟空).pdf', created_at=1724295950, word_count=0, enabled=True, meta=DocumentMeta(source='upload_file', file_id='fc170b9e-0876-4d82-903c-5976e9a58453')), Document(id='dfeb440e-dc5f-4d1a-b0d8-6561b9fb5786', name='https://baijiahao.baidu.com/s?id=1802527379394162441', created_at=1724295904, word_count=0, enabled=True, meta=DocumentMeta(source='url', file_id=None))]
get_all_documents
id='139be427-5024-4af7-b5ca-1f15efdfea86' name='黑神话(悟空).pdf' created_at=1724295950 word_count=0 enabled=True meta=DocumentMeta(source='upload_file', file_id='fc170b9e-0876-4d82-903c-5976e9a58453')
id='dfeb440e-dc5f-4d1a-b0d8-6561b9fb5786' name='https://baijiahao.baidu.com/s?id=1802527379394162441' created_at=1724295904 word_count=0 enabled=True meta=DocumentMeta(source='url', file_id=None)


- 现在我们可以在Appbuilder中查看我们的知识库

![向知识库传入文档](https://bj.bcebos.com/v1/appbuilder-sdk-components/%E5%90%91%E7%9F%A5%E8%AF%86%E5%BA%93%E4%BC%A0%E5%85%A5%E6%96%87%E6%A1%A3.png?authorization=bce-auth-v1%2FALTAKGa8m4qCUasgoljdEDAzLm%2F2024-08-22T10%3A56%3A50Z%2F-1%2Fhost%2Fe0a39a3fd3d2acd0f4b949d68b895b2943c63f94db7813fcace9674a11290681)

# 现在我们基于new_test_knowledge知识库创建了Client应用

![创建黑神话Client](https://bj.bcebos.com/v1/appbuilder-sdk-components/%E9%BB%91%E7%A5%9E%E8%AF%9DClient.png?authorization=bce-auth-v1%2FALTAKGa8m4qCUasgoljdEDAzLm%2F2024-08-22T02%3A19%3A30Z%2F-1%2Fhost%2F93016fe0808f5460859d678d6b2095ee6624842db914e15c73bd31a476b42402)

接下来我们调用Client应用，查看运行效果

In [22]:
# 从AppBuilder控制台【个人空间】-【应用】网页获取已发布应用的ID
app_id = "236b3e57-3464-43b0-8556-f165193721ef"

app_builder_client = appbuilder.AppBuilderClient(app_id)
conversation_id = app_builder_client.create_conversation()

resp = app_builder_client.run(conversation_id, "请你用50字左右介绍一下《黑神话(悟空)》")
print(resp.content.answer)

《黑神话：悟空》是一款充满文化底蕴的游戏，用现代方式讲述经典中国故事，引发全球用户的共鸣^[1]^。


##### KnowledgeBase 同样提供代码态的接口，可以对知识库文档进行切片

* 创建切片
* 修改切片信息
* 获取切片信息
* 获取切片列表
* 删除切片

- 首先我们尝试让Client回答KnowledgeBase中没有的信息

In [67]:
# 从AppBuilder控制台【个人空间】-【应用】网页获取已发布应用的ID
app_id = "236b3e57-3464-43b0-8556-f165193721ef"

app_builder_client = appbuilder.AppBuilderClient(app_id)
conversation_id = app_builder_client.create_conversation()

resp = app_builder_client.run(conversation_id, "《黑神话：悟空》的‌虎先锋怎么打？能不能提供一下攻略？")
print(resp.content.answer)

很抱歉，我没有找到关于《黑神话：悟空》中虎先锋的具体打法攻略。但是，一般来说，在动作冒险游戏中，打败敌人的关键通常在于熟练掌握角色的攻击方式、防御技巧和敌人的攻击模式。你可以尝试多次与虎先锋战斗，观察它的攻击方式和弱点，并结合孙悟空的技能和装备来制定相应的战斗策略。此外，你也可以在游戏的官方论坛或社区中寻找其他玩家分享的攻略和经验，这可能会对你有所帮助。

请注意，不同玩家的游戏风格和策略可能有所不同，因此建议你在参考他人攻略的同时，结合自己的游戏情况进行调整和优化。


- 这时候我们就需要使用为KnowledeBase添加切片为他增添知识点了
###### 创建切片`create_chunk`

- 为了增加对照，我们先为Client添加BaiduSearch组件，并运行它查看他能提供哪些信息

In [68]:
# 从AppBuilder控制台【个人空间】-【应用】网页获取已发布应用的ID
app_id = "236b3e57-3464-43b0-8556-f165193721ef"

app_builder_client = appbuilder.AppBuilderClient(app_id)
conversation_id = app_builder_client.create_conversation()

resp = app_builder_client.run(conversation_id, "《黑神话：悟空》的‌虎先锋怎么打？能不能提供一下攻略？")
print(resp.content.answer)

《黑神话：悟空》的‌虎先锋打法攻略如下^[2]^：

1. 起手进行蓄力，第一次蓄力攻击和虎先锋换血。
2. 闪身躲避虎先锋的两端攻击。
3. 对其进行攻击，使用精魄将其打到硬直，接着使用定身术将其定住，然后再进行攻击。
4. 注意虎先锋会开启石化技能，可以规避攻击，并且进行反击，玩家在其进入石化后应避免出招。
5. 在其第二次石化之后拉开距离，观察情况，躲避其的突进。
6. 当虎先锋掏出剑之后，注意躲避其攻击，等待其将剑收回去再进行攻击。
7. 在虎先锋血量很少的情况下使用定身来进行输出，随后使用攒的棍势再次输出即可击败虎先锋。


- 我们发现BaiduSearch提供的信息不一定符合我们预期，这个时候我们就需要对知识库增加新的切片，来为我们的知识库不断更新知识点，获取document_id,支持对文档进行切片操作

```python
# 获取doc_list列表的第一个文档的document_id
document_id = doc_list[0].id
```

- content: 切片内容

In [69]:
# 获取doc_list列表的第一个文档的document_id
document_id = doc_list[0].id
my_knowledge = appbuilder.KnowledgeBase(my_knowledge_base_id)
print("知识库ID: ", my_knowledge.knowledge_id)
# 传入了一个内容为"content"的切片
content = """
虎先锋是黑神话悟空中的一个boss，很多的玩家都非常的好奇这个boss要怎么样打，这里小编就为大家对虎先锋这个boss的打法进行了整合，可以让你更好的去进行战斗，帮助你击败这个敌人，详细的内容就在这里，快来一起看看吧。
1、进入战斗后注意躲避boss的坠击伤害，快速闪避即可。
2、和BOSS保持一定的距离，如果距离过近，它会对玩家造成三段连击伤害。
3、当BOSS释放吼叫后，直接远离它，在远程进行输出。
4、然后使用闪避躲避它释放的三连剑气和黑色的气旋。
5、最后当BOSS变的坚硬时不作攻击，等它技能结束后继续输出即可轻松过关。
"""
resp = my_knowledge.create_chunk(documentId=document_id, content=content)
print("切片ID: ", resp.id)

知识库ID:  3cc48493-f9ff-48c8-a3ea-fcc17fdddded
切片ID:  19e93596-a314-4ca3-81fa-d6c041433979


- 再次运行Client看看他能否输出我们预期的知识点

In [70]:
app_id = "236b3e57-3464-43b0-8556-f165193721ef"

app_builder_client = appbuilder.AppBuilderClient(app_id)
conversation_id = app_builder_client.create_conversation()

resp = app_builder_client.run(conversation_id, "《黑神话：悟空》的‌虎先锋怎么打？能不能提供一下攻略？")
print(resp.content.answer)

《黑神话：悟空》的‌虎先锋打法攻略如下^[1][3]^：

1. 起手进行蓄力，第一次蓄力攻击和虎先锋换血。

2. 随后闪身躲避起手的两端攻击。

3. 对其进行攻击，使用精魄将其打到硬直，接着使用定身法将其定住，然后再进行攻击。

4. 注意虎先锋会开启石化技能，可以规避攻击，并且进行反击，玩家在其进入石化后应尽可能避免出招。

5. 在虎先锋血量很少的情况下使用定身法来进行输出，随后再输出即可击败虎先锋。


- 这次输出的信息就符合我们预期了。

###### 修改切片信息`modify_chunk`

- 如果我们对之前设置的切片信息不满意，可以通过`modify_chunk`方法进行修改。

###### 获取切片信息`describe_chunk`

- 我需要查看我刚才创建切片信息，可以使用`describe_chunk`方法

###### 删除切片信息`delete_chunk`

- 切片信息创建出来后，如果不需要了，可以通过`delete_chunk`方法进行删除。

##### 删除文档和知识库

- 当你不再需要文档知识库时，可以删除它，SDK同样提供删除文档和知识库的方法
  - 从知识库删除文档 `KnowledgeBase().delete_document`
  - 删除知识库`delete_knowledge_base`